런타임에서 GPU 연결 꼭 하세요!

#1. Install&Import

In [ ]:
!pip install --upgrade diffusers accelerate transformers openai

In [ ]:
import openai
api_key=input("Enter your OpenAI API key: ") #여러분들의 openai api key를 입력해주세요!
openai.api_key=api_key

#2. ChatGPT: Prompt->Keyword

In [ ]:
# bookname='Little Prince'
# bookname='Crime and Punishment'
bookname='1984'
# bookname='brave new world'
# bookname='Wuthering Heights'

In [ ]:
def chatGPT(bookname):
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", 
             "content": f"Write book keywords follwing below templete. Book title is {bookname}.\n\
             Title : Write book title, except book author\n\
             Character : List 2 character keywords in one line\n\
             Object : List 2 object keywords in one line\n\
             Atmosphere : List 3 atmosphere keywords in one line"}
        ]
    )
    result=completion.choices[0].message["content"]
    return result

In [ ]:
message=chatGPT(bookname)

In [ ]:
print(message)
# 정상적으로 작동할 때의 결과는 다음과 같았습니다.

# Title : Little Prince
# Character: prince, pilot
# Object: rose, fox
# Atmosphere: whimsical, philosophical

#3. Preprocess: Keyword->SD Prompt

In [ ]:
def preprocess(message):
    splited=message.split(':')
    def slicing(sentence):
        num=sentence.find('\n')
        return sentence[1:num]
    sliced=list(map(slicing,splited))[1:]
    result=','.join(sliced)
    return result

In [ ]:
bookprompt=preprocess(message)
positivePrmt="masterpiece, 8k, extremely detailed"
negativePrmt="disfigured, kitsch, low-res, Deformed, blurry, out of focus, disgusting, poorly drawn, (watermark), (nsfw:2), (sexual content:2)"
# print(positivePrmt+',',bookprompt)

#4. Stable Diffusion: SD Prompt->Image

In [ ]:
from diffusers import DiffusionPipeline
def SD(bookprompt,positivePrmt,negativePrmt):
    repo_id = "sinkinai/cheese-daddys-landscapes-mix"
    pipeline = DiffusionPipeline.from_pretrained(repo_id,safety_checker=None).to("cuda")
    images = pipeline(num_inference_steps=30,
                 prompt=positivePrmt+','+bookprompt,
                 negative_prompt=negativePrmt,
                 num_images_per_prompt=8).images
    return images

In [ ]:
images=SD(bookprompt,positivePrmt,negativePrmt)

#5. Show Images

In [ ]:
from PIL import Image

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

In [ ]:
grid=image_grid(images,rows=2,cols=4)

In [ ]:
grid